#Import libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sqlalchemy import create_engine, text
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
import re


#Create the database manually

In [ ]:
engine = create_engine("sqlite:///sample.db")
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS sales"))
    conn.execute(text("""
        CREATE TABLE sales (
            id INTEGER PRIMARY KEY,
            product TEXT,
            amount INTEGER
        )
    """))
    conn.execute(text("""
        INSERT INTO sales (product, amount) VALUES
        ('Laptop', 1200),
        ('Phone', 800),
        ('Tablet', 600),
        ('Phone', 900),
        ('Laptop', 1500)
    """))


#Import the llm model

In [ ]:
db = SQLDatabase(engine)
hf_pipeline = pipeline(
    "text-generation",
    model="defog/sqlcoder-7b-2",
    device=0,
    torch_dtype="auto",
    temperature=0.0,
    max_new_tokens=100,
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-1076702355.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


# Create chain

In [ ]:
chain = create_sql_query_chain(llm, db)

#Extract the output from the chain to plain text

In [ ]:
def extract_sql(text):
    # Try ```sql fenced block
    m = re.search(r"```sql(.*?)```", text, flags=re.S|re.I)
    if m:
        return m.group(1).strip()
    # Try plain SELECT ... ;
    m = re.search(r"(select .*?);", text, flags=re.S|re.I)
    if m:
        return m.group(0).strip()
    return text.strip()


Ask the Question to Database


In [ ]:
question = "Which product has the highest total sales?"
raw_output = chain.invoke({"question": question})

# Extract clean SQL
sql = extract_sql(raw_output)
print("🔹 Generated SQL:\n", sql)

# 6. Run SQL safely
result = db.run(sql)
print("🔹 Result:\n", result)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔹 Generated SQL:
 SELECT s.product, SUM(s.amount) AS total_sales FROM sales s GROUP BY s.product ORDER BY total_sales DESC NULLS LAST LIMIT 1;
🔹 Result:
 [('Laptop', 2700)]


#conclusion

In this project, we transformed a small sales dataset into a normalized relational database consisting of two tables, products and sales. The products table stores unique product details such as name and base price, while the sales table records individual transactions and links them back to the corresponding product. This structured schema provided a strong foundation for querying and analysis. To enable natural language interaction with the database, we integrated the system with the Hugging Face model defog/sqlcoder-7b-2 through a LangChain pipeline. This pipeline allows plain English questions, such as “Which product has the highest total sales?”, to be automatically converted into executable SQL queries, which are then run against the SQLite database to return results. Using this approach, we discovered that Laptops contributed the highest revenue, Phones and Laptops showed variability in pricing, and Tablets were the least frequent and least profitable product. Overall, this project demonstrates how a small dataset can be structured, expanded, and leveraged through LangChain and Hugging Face LLMs to bridge the gap between natural language and SQL, making data exploration more accessible and intuitive.